This is the first colab notebook of the GlyphGAN model that Henry and I made for our final of Allison's [Material of Language](https://mol.decontextualize.com/) class. It prepares the font data set for the second notebook (GlyphGAN_Train) so that we can train the GlyphGAN model.

The code is heavily based on this repo: https://github.com/thunchakorn/ThaiFontGANs

# Getting Started
The model that we're using, GlyphGAN, is a style-consistent font generator. You can feed the model with glyph images from a number of existing fonts, and then generate the same set of glyphs with brand new fonts. We selected a number of Chinese calligraphy fonts, and took the characters from an ancient peom to use it as the dataset.

The poem, titled "望庐山瀑布", was written by Li Bai, a famous Chinese poet of the Tang Dynasty. It desribes the magnificent scene of a waterfall in southern China.


In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from PIL import ImageFont, Image, ImageDraw, ImageChops, ImageOps
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import json

In [0]:
fonts_directory = '/content/gdrive/My Drive/2020 Spring/Material of Language/MOL Final/Fonts/Calligraphy/'

In [4]:
fonts = os.listdir(fonts_directory)
image_size = 128
chars = '日照香炉生紫烟遥看瀑布挂前川飞流直下三千尺疑是银河落九天，。' # characters from the poem, "望庐山瀑布"

print(len(fonts), len(chars))

15 30


Then two helper functions were defined, one to turn font data into an array, the other plot the chosen characters with the specified font and save it as an image.

In [0]:
def font2array(font_ttf, chars, size = 64, ratio = 0.5):
    blank = Image.new('L', (size*4, size*4), 255)
    font = ImageFont.truetype(font_ttf, int(size))
    # We need to make sure we scale down the fonts but preserve the vertical alignment
    min_ly = float('inf')
    max_hy = float('-inf')
    max_width = 0
    imgs = []

    for char in chars:
        # Draw character
        img = Image.new("L", (size*3, size*3), 255)
        draw = ImageDraw.Draw(img)
        w, h = font.getsize(char)
        draw.text(((size*3-w)//2,(size*3-h)//2), char, font=font)
        # Get bounding box
        diff = ImageChops.difference(img, blank)
        if diff.getbbox() == None:
            print(f'char:{char} of font:{font_ttf}')
            return
        lx, ly, hx, hy = diff.getbbox()
        min_ly = min(min_ly, ly)
        max_hy = max(max_hy, hy)
        max_width = max(max_width, hx - lx)
        imgs.append((lx, hx, img))

    #print('crop dims:', max_hy - min_ly, max_width)
    scale_factor = min(ratio * size / (max_hy - min_ly), ratio * size / max_width)
    data = []

    for lx, hx, img in imgs:
        img = img.crop((lx, min_ly, hx, max_hy))

        # Resize to smaller
        new_width = (hx-lx) * scale_factor
        new_height = (max_hy - min_ly) * scale_factor
        img = img.resize((int(new_width), int(new_height)), Image.ANTIALIAS)

        # Expand to square
        img_sq = Image.new('L', (size, size), 255)
        offset_x = (size - new_width)/2
        offset_y = (size - new_height)/2
        img_sq.paste(img, (int(offset_x), int(offset_y)))

        # Convert to numpy array
        matrix = np.array(img_sq.getdata()).reshape((size, size))
        matrix = 255 - matrix
        data.append(matrix)

    return np.array(data)

In [0]:
def plot_font(data, font_name):
    fig = plt.figure(figsize=(9, 6))
    for i in range(data.shape[0]):
      plt.subplot(9, 6, i+1)
      plt.axis('off')
      plt.imshow(255-data[i], cmap = 'gray')
      plt.savefig(f'plot_font/{font_name[:-4]}.png', bbox_inches='tight')
    plt.clf()

In [0]:
!mkdir check_font

In [0]:
#checking for courrupt fonts. 
for font in fonts:
    test = font2array(font_ttf = fonts_directory + font, chars = '鸡', size = image_size, ratio = 0.8)
    if test is None:
        continue
    img = Image.fromarray(test[0].astype(np.uint8))
    img.save(f'check_font/{font[:-4]}.PNG', 'PNG')

In [0]:
!mkdir plot_font

In [13]:
for i, font in zip(range(data.shape[0]), fonts):
    plot_font(data[i], font)
    if i % 10 == 0:
        print(f'finish {i}')

finish 0
finish 10


<Figure size 648x432 with 0 Axes>

<Figure size 648x432 with 0 Axes>

<Figure size 648x432 with 0 Axes>

<Figure size 648x432 with 0 Axes>

...

<Figure size 648x432 with 0 Axes>

<Figure size 648x432 with 0 Axes>

<Figure size 648x432 with 0 Axes>

<Figure size 648x432 with 0 Axes>

<Figure size 648x432 with 0 Axes>

Then we can create the dataset for training by loading the characters of the peom from each font, and save them as a npy file. This npy file will be used to train the model.

In [14]:
fonts = os.listdir(fonts_directory)
num_fonts = len(fonts)
num_chars = len(list(chars))

# numpy array data
data = np.array([])

report_interval = 10
i = 0

print(f'#font:{len(fonts)}\n#char:{num_chars}')
start = time.time()

for font in fonts:
    char_font = font2array(font_ttf = fonts_directory + font, chars = chars, size = image_size, ratio = 0.9)
    data = np.append(data, char_font)
    if i % report_interval == 0:
        print(f'finish font:{i+1}th')
    i += 1

print(f'time use: {time.time()-start}')

print(data.shape)
data = data.astype(np.uint8)
data = data.reshape(-1, num_chars, image_size, image_size)
print(data.shape)

#font:15
#char:30
finish font:1th
finish font:11th
time use: 2.382444381713867
(7372800,)
(15, 30, 128, 128)


In [0]:
np.save('fontarray_calligraphy.npy', data)